In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import pandas as pd
import datetime
import operator
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

/home/zhangxs/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import sys
sys.path.append('/home/zhangxs/soft/LightGBM/python-package/')
import lightgbm as lgb

In [7]:
df_train = pd.read_csv("./../quora_pairs/data/new_feature/train_final.csv")
x_train = df_train.values[:,2:]
print 'x_train.shape',x_train.shape

train_x.shape (404290, 81)


In [8]:
df_train_src = pd.read_csv("./../quora_pairs/data/train.csv")
y_train = df_train_src["is_duplicate"].values
print "y_train",y_train.shape

train_y (404290,)


[Tutorial](https://github.com/Microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py)

In [22]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.1)
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval = lgb.Dataset(X_val, Y_val, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'binary_error'},
    'metric_freq':10,
    'num_leaves': 35,
    'learning_rate': 0.08,
    'max_depth':5,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'lambda_l2':0.005,
    'bagging_freq': 10,
    'verbose': 1,
#     'device':'gpu',
}

In [23]:
bst = lgb.train(params, 
                lgb_train, 
                num_boost_round=1000,
                valid_sets=[lgb_train,lgb_eval],
                early_stopping_rounds=50)

[1]	training's binary_logloss: 0.658037	training's binary_error: 0.223794	valid_1's binary_logloss: 0.657981	valid_1's binary_error: 0.222316
Train until valid scores didn't improve in 50 rounds.
[2]	training's binary_logloss: 0.627698	training's binary_error: 0.20892	valid_1's binary_logloss: 0.627483	valid_1's binary_error: 0.207079
[3]	training's binary_logloss: 0.602672	training's binary_error: 0.204559	valid_1's binary_logloss: 0.60235	valid_1's binary_error: 0.20327
[4]	training's binary_logloss: 0.580254	training's binary_error: 0.201418	valid_1's binary_logloss: 0.580012	valid_1's binary_error: 0.199609
[5]	training's binary_logloss: 0.550025	training's binary_error: 0.163439	valid_1's binary_logloss: 0.550107	valid_1's binary_error: 0.165203
[6]	training's binary_logloss: 0.523545	training's binary_error: 0.151027	valid_1's binary_logloss: 0.523893	valid_1's binary_error: 0.152935
[7]	training's binary_logloss: 0.508388	training's binary_error: 0.152624	valid_1's binary_loglos

## FOR TEST

In [ ]:
pred_probs = bst.predict(x_test,num_iteration=bst.best_iteration)

In [ ]:
df_test_src = pd.read_csv("./../quora_pairs/data/test.csv")

In [ ]:
print("Writing output...")
sub = pd.DataFrame()
sub['test_id'] = df_test_src['test_id']
sub['is_duplicate'] = pred_probs #*.75
sub.to_csv("lgb_with_feature.csv", index=False)